In [11]:
# !pip install openpyxl

In [12]:
# ============================================================
# 🧭 Y06_안전비상벨_TM 주소 표준화 전처리 (서울특별시 고정 버전)
# - ID 정리 (공백/탭 제거)
# - 중복 주소 컬럼 통합
# - 시 = '서울특별시' 고정 + 구/동/도로명 분리
# - 코드형 컬럼 정제 (PNU/행정동/법정동/우편번호 등) → .0 및 지수 제거
# - 좌표(위도/경도/X/Y)만 float, 자릿수 고정
# - 컬럼 순서 정리 후 저장
# ============================================================


import pandas as pd
import re
import os
from IPython.display import display

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y06_안전비상벨_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/raw/preprocessing"
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 자동 생성

output_file = os.path.join(output_dir, "Y06_안전비상벨_전처리.csv")

# ===== 1) 파일 로드 + NaN 확인 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 파일 로드 완료: {df.shape}\n")
print("📊 데이터 기본 정보 (NaN 확인 포함):")
df.info()  # 결측치 확인용


✅ 파일 로드 완료: (1000, 39)

📊 데이터 기본 정보 (NaN 확인 포함):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _OBJT_ID    1000 non-null   object 
 1   FCLTY_TY    908 non-null    object 
 2   MNG_INST    908 non-null    object 
 3   INS_PURPOS  908 non-null    object 
 4   INS_TYPE    908 non-null    object 
 5   INS_DETAIL  908 non-null    object 
 6   RN_ADRES    908 non-null    object 
 7   ADRES       908 non-null    object 
 8   LAT         908 non-null    float64
 9   LON         908 non-null    float64
 10  LNK_TYPE    908 non-null    object 
 11  FLAG_POL_L  908 non-null    object 
 12  FLAG_SEC_L  908 non-null    object 
 13  FLAG_MNG_L  908 non-null    object 
 14  ADDITION    908 non-null    object 
 15  INS_YEAR    908 non-null    float64
 16  LAST_INSPD  908 non-null    float64
 17  LAST_INSPT  908 non-null    object 
 18  MNG_TEL     908 non-

In [13]:
# ===== 2) 항목정의 기반 컬럼 매핑 =====
col_map = {
    "OBJT_ID": "일련번호",
    "FCLTY_TY": "유형명",
    "MNG_INST": "관리기관명",
    "INS_PURPOS": "설치목적",
    "INS_TYPE": "설치장소유형",
    "INS_DETAIL": "설치위치",
    "RN_ADRES": "도로명주소",
    "ADRES": "지번주소",
    "LAT": "위도",
    "LON": "경도",
    "LNK_TYPE": "연계방식",
    "FLAG_POL_L": "경찰연계유무",
    "FLAG_SEC_L": "경비업체연계유무",
    "FLAG_MNG_L": "관리사무소연계유무",
    "ADDITION": "부가기능",
    "INS_YEAR": "안전비상벨설치연도",
    "LAST_INSPD": "최종점검일자",
    "LAST_INSPT": "최종점검결과구분",
    "MNG_TEL": "관리기관전화번호",
    "FLAG_SERVI": "연계유무",
    "CTPRVN_CD": "시도코드",
    "SGG_CD": "시군구코드",
    "EMD_CD": "읍면동코드",
    "X": "X좌표",
    "Y": "Y좌표",
    "DATA_TY": "데이터기준일자"
}
df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)
print("\n✅ 컬럼명 한글 변환 완료")
print(df.columns.tolist())

# ===== 3) 좌표만 숫자형 변환 + 자릿수 고정 =====
def format_coords(x, decimals):
    if pd.isna(x) or x == "":
        return None
    try:
        return round(float(x), decimals)
    except:
        return pd.to_numeric(x, errors="coerce")

for col, dec in [("위도", 6), ("경도", 6), ("X좌표", 4), ("Y좌표", 5)]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].apply(lambda v: format_coords(v, dec))

# ===== 4) '.0' & 지수표기 제거: 코드/날짜 컬럼 표준화 =====
def to_digit_str(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    s = str(x).strip()
    if re.fullmatch(r"\d+\.0+", s):
        s = s.split(".")[0]
    try:
        if re.search(r"[eE]\+?", s):
            f = float(s)
            s = str(int(f)) if f.is_integer() else re.sub(r"[^\d]", "", s)
    except:
        pass
    s = re.sub(r"[^\d]", "", s)
    return s if s else None

fix_cols = ["안전비상벨설치연도", "최종점검일자", "데이터기준일자",
            "행정동코드", "법정동코드", "우편번호", "시도코드", "시군구코드", "읍면동코드"]
for c in fix_cols:
    if c in df.columns:
        df[c] = df[c].apply(to_digit_str)

for pnu_col in ["PNU", "PUN"]:
    if pnu_col in df.columns:
        df[pnu_col] = df[pnu_col].apply(to_digit_str)

# ===== 5) 주소 파싱 유틸 (서울특별시 고정 + 구/군만 구로 인정) =====
def _normalize_tokens(addr: str):
    # 공백 분할 후, 앞뒤 특수문자 제거(콤마/괄호 등)
    raw = re.split(r"\s+", str(addr).strip())
    toks = [re.sub(r"^[^\w가-힣]+|[^\w가-힣]+$", "", t) for t in raw if t]
    return [t for t in toks if t]

def smart_parse(addr):
    # 반환: (시, 구, 동, 도로명) — 시는 항상 '서울특별시'
    if pd.isna(addr) or str(addr).strip() == "":
        return "서울특별시", None, None, None

    t = _normalize_tokens(addr)
    if not t:
        return "서울특별시", None, None, None

    # 1) '서울특별시' 토큰은 건너뛰기
    i = 1 if t[0] == "서울특별시" or (t[0].endswith("시") and "서울" in t[0]) else 0

    # 2) 구 찾기: '구' 또는 '군'으로 끝나는 첫 토큰
    gu = None
    while i < len(t) and not re.search(r"(구|군)$", t[i]):
        i += 1
    if i < len(t):
        gu = t[i]
        i += 1

    # 3) 동 추정: 다음 토큰이 '동' 포함/종료(예: 청운효자동, 삼성동)
    dong = None
    if i < len(t):
        if re.search(r"동$", t[i]) or ("동" in t[i]):  # 행정동명은 보통 '동' 포함
            dong = t[i]
            i += 1

    # 4) 나머지를 도로명(또는 번지 포함 주소)로
    road = " ".join(t[i:]) if i < len(t) else None

    return "서울특별시", gu, dong, road

# ===== 6) 행정동명 → 시/구/동/도로명 초기 채움 =====
if "행정동명" not in df.columns:
    raise ValueError("❌ '행정동명' 컬럼이 없습니다.")
df[["시", "구", "동", "도로명"]] = df["행정동명"].apply(lambda x: pd.Series(smart_parse(x)))

# ===== 7) 도로명주소/지번주소로 결측 보완 (같은 파서 재사용) =====
def fill_from_col(row, colname):
    val = row.get(colname)
    if pd.notna(val) and str(val).strip() != "":
        _, g, d, r = smart_parse(val)
        if (row.get("구") in [None, ""]) and g:   row["구"] = g
        if (row.get("동") in [None, ""]) and d:   row["동"] = d
        if (row.get("도로명") in [None, ""]) and r: row["도로명"] = r
    return row

for src_col in ["도로명주소", "지번주소"]:
    if src_col in df.columns:
        df = df.apply(fill_from_col, colname=src_col, axis=1)

# ===== 8) 시/구/동/도로명 컬럼을 '유형명' 다음으로 이동 =====
target_cols = ["시", "구", "동", "도로명"]
cols = list(df.columns)
type_idx = cols.index("유형명") if "유형명" in cols else 0
cols = [c for c in cols if c not in target_cols]
new_cols = cols[:type_idx+1] + target_cols + cols[type_idx+1:]
df = df[new_cols]

print("\n✅ 시/구/동/도로명 컬럼 이동 완료")
print(df.columns.tolist())

# ===== 9) 저장 및 미리보기 =====
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n💾 전처리 완료 CSV 저장: {output_file}")
display(df.head(10))


✅ 컬럼명 한글 변환 완료
['_OBJT_ID', '유형명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']

✅ 시/구/동/도로명 컬럼 이동 완료
['_OBJT_ID', '유형명', '시', '구', '동', '도로명', '관리기관명', '설치목적', '설치장소유형', '설치위치', '도로명주소', '지번주소', '위도', '경도', '연계방식', '경찰연계유무', '경비업체연계유무', '관리사무소연계유무', '부가기능', '안전비상벨설치연도', '최종점검일자', '최종점검결과구분', '관리기관전화번호', '연계유무', '시도코드', '시군구코드', '읍면동코드', 'X좌표', 'Y좌표', '데이터기준일자', '입력주소', 'X.1', 'Y.1', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']

💾 전처리 완료 CSV 저장: /Users/mac/Documents/SORA_Project/data/raw/preprocessing/Y06_안전비상벨_전처리.csv


,_OBJT_ID,유형명,시,구,동,도로명,관리기관명,설치목적,설치장소유형,설치위치,...,CLSS,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명
0,1,비상벨,서울특별시,종로구,청운동,7-3,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구 서시정),...,정좌표,1111010100100070016,구주소,NaN,서울특별시 종로구 청운동 7-3,None,1111051500,청운효자동,1111010100,청운동
1,2,비상벨,서울특별시,종로구,청운동,7-3,종로구청,방범용,공원,인왕산도시자연공원,...,정좌표,1111010100100070016,구주소,NaN,서울특별시 종로구 청운동 7-3,None,1111051500,청운효자동,1111010100,청운동
2,3,비상벨,서울특별시,종로구,청운동,7-27,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구),...,정좌표,1111010100100070016,구주소,NaN,서울특별시 종로구 청운동 7-27,None,None,NaN,1111010100,청운동
3,4,비상벨,서울특별시,종로구,청운동,산 1-1,종로구청,약자보호,화장실,창의문화장실,...,정좌표,1111010100200009984,구주소,서울특별시 종로구 창의문로 42,서울특별시 종로구 청운동 산1-1,3048,1111051500,청운효자동,1111010100,청운동
4,5,비상벨,서울특별시,종로구,청운동,산 1-1,종로구청,약자보호,화장실,창의문화장실,...,정좌표,1111010100200009984,구주소,서울특별시 종로구 창의문로 42,서울특별시 종로구 청운동 산1-1,3048,1111051500,청운효자동,1111010100,청운동
5,6,비상벨,서울특별시,종로구,궁정동,55-3,종로구청,약자보호,화장실,무궁화동산,...,정좌표,1111010300100550016,구주소,NaN,서울특별시 종로구 궁정동 55-3,None,1111051500,청운효자동,1111010300,궁정동
6,7,비상벨,서울특별시,종로구,누상동,산 1-3,종로구청 청소행정과,방범용,화장실,누상동체육시설 공중화장실,...,정좌표,1111010900200009984,구주소,서울특별시 종로구 옥인6길 26-17,서울특별시 종로구 누상동 산1-3,3038,1111051500,청운효자동,1111010900,누상동
7,8,비상벨,서울특별시,종로구,누상동,산 1-38,종로구청,약자보호,화장실,인왕산도시자연공원(누상지구),...,정좌표,1111010900200009984,구주소,NaN,서울특별시 종로구 누상동 산1-38,None,1111051500,청운효자동,1111010900,누상동
8,9,비상벨,서울특별시,종로구,옥인동,185-4,종로구청,방범용,공원,수성동계곡,...,정좌표,1111011100101849984,구주소,NaN,서울특별시 종로구 옥인동 185-4,None,None,NaN,1111011100,옥인동
9,10,비상벨,서울특별시,종로구,옥인동,179-1,종로구청 청소행정과,방범용,화장실,인왕산수목원약수터 공중화장실,...,정좌표,1111011100101789952,구주소,NaN,서울특별시 종로구 옥인동 179-1,3034,1111051500,청운효자동,1111011100,옥인동
